In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [ ]:
df = pd.read_csv("service_price_dataset.csv")
X = df.drop('price', axis=1)
y = df['price']


In [ ]:
categorical_features = ['category', 'difficulty']
encoder = OneHotEncoder(sparse_output=False)
X_encoded = encoder.fit_transform(X[categorical_features])

In [ ]:
numerical_features = ['short_notice', 'weekend', 'distance_km', 'review_score', 'past_jobs']
X_numerical = X[numerical_features].values
X_final = np.hstack([X_encoded, X_numerical])

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_final)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [13]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)  # Regression output
])


C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
history = model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=32, verbose=1)
loss, mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {mae:.2f}")

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 41348.4766 - mae: 191.6389 - val_loss: 42591.0234 - val_mae: 194.7373
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 40659.0586 - mae: 189.8323 - val_loss: 41652.7070 - val_mae: 192.3349
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 39487.1523 - mae: 186.8062 - val_loss: 40106.1094 - val_mae: 188.3599
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 37592.5430 - mae: 181.7920 - val_loss: 37758.7500 - val_mae: 182.2348
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 34986.5703 - mae: 174.6992 - val_loss: 34464.9609 - val_mae: 173.4243
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 31269.3340 - mae: 164.4116 - val_loss: 30232.0527 - val_mae: 161.5223
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 26638.7031 - mae: 150.8185 - val_loss: 25065.5469 - val_mae: 145.9362
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 21305.3965 - mae: 133.7541 - val_loss

In [15]:
def predict_price(category, difficulty, short_notice, weekend, distance_km, review_score, past_jobs):
    cat_encoded = encoder.transform([[category, difficulty]])
    num_features = np.array([[short_notice, weekend, distance_km, review_score, past_jobs]])
    X_input = np.hstack([cat_encoded, num_features])
    X_input_scaled = scaler.transform(X_input)
    return float(model.predict(X_input_scaled, verbose=0))


In [ ]:
example_price = predict_price("cleaning", "high", 1, 0, 15, 4.5, 50)
print(f"Predicted price: {example_price:.2f} TND")

Predicted price: 124.19 TND


C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
C:\Users\ASUS\AppData\Local\Temp\ipykernel_20104\3702384168.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(model.predict(X_input_scaled, verbose=0))


: 